## Inferencia causal: ejemplos introductorios

In [ ]:
%autosave 0
import pandas as pd 
import numpy as np
from plotnine import *
import plotnine

### Datos

In [ ]:
programa = pd.read_table("../datos/electric.dat", sep = "\s+")
programa


En este caso:

- Para cada ciudad y para cada grado, se escogió un número chico de escuelas (10-20)
- Dentro de cada escuela, se escogieron dos salones del mismo grado, y se asignó al azar una
a tratamiento (el grupo ve el programa de tv para aprender a leer) o a control (el grupo sigue con los 
ejercicios de lectura usuales)

Nota: hay un detalle adicional de este experimento que discutiremos más adelante. Por el momento
haremos el análisis estándar según la explicación anterior.

Veamos un resumen de las calificaciones por grado:

In [ ]:
programa.groupby("Grade").agg({"treated.Posttest":[np.mean], "control.Posttest":[np.mean]}).round(3)


Preparamos los datos para correr una regresión:

In [ ]:
nombres_1 = {"treated.Posttest":"resultado", "treated.Pretest":"calificacion_pre"}
nombres_2 = {"control.Posttest":"resultado", "control.Pretest":"calificacion_pre"}
tratados = programa[["Grade", "treated.Posttest", "treated.Pretest"]].rename(columns = nombres_1)
no_tratados = programa[["Grade", "control.Posttest", "control.Pretest"]].rename(columns = nombres_2)
tratados["tratamiento"] = 1
no_tratados["tratamiento"] = 0
programa_larga = pd.concat([tratados, no_tratados], axis = 0)
programa_larga

In [ ]:
# Corremos el modelo para cada grado
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
coefs = []  
for g in [1,2,3,4]:
  datos_grado = programa_larga[programa_larga.Grade == g]
  y = datos_grado["resultado"].to_numpy()
  #dummis_grado = pd.get_dummies(programa_larga.Grade, prefix = "Grado_", drop_first=True)
  datos_p = datos_grado.drop(columns = ["Grade", "resultado", "calificacion_pre"])
  #datos_p = pd.concat([dummis_grado, datos_p], axis = 1, sort=False)
  datos_p["const"] = 1
  reg_1 = sm.OLS(y, datos_p).fit()
  intervalos = reg_1.conf_int()
  params = reg_1.params
  resumen = pd.concat([params, intervalos], axis = 1)
  resumen.columns = ["coef", "inf", "sup"]
  resumen["Grado"] = g
  coefs.append(resumen)
resumen = pd.concat(coefs)
resumen.round(2)

Interpreta estos coeficientes (tratamiento y constante). Podemos graficar:

In [ ]:
coef_trata =  resumen[resumen.index == "tratamiento"]
(ggplot(coef_trata, aes("Grado", "coef", ymin="inf", ymax="sup")) +
  geom_hline(yintercept = 0) +
   geom_linerange(colour = "red") +
   geom_point(size = 3) + coord_flip())

¿Puedes interpretar estos coeficientes de manera causal? ¿Por qué si o no?

### Agregando covariables pretratamiento


Aunque en este ejemplo el efecto no está confundido, podemos usar covariables
pre-tratamiento para intentar explicar mejor la variación de los resultados, y obtener
estimaciones más precisas de los efectos del programa. Controlamos entonces por las
calificaciones de lectura que tomamos *antes* de comenzar el experimento:


In [ ]:
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
coefs = []  
for g in [1,2,3,4]:
  datos_grado = programa_larga[programa_larga.Grade == g]
  y = datos_grado["resultado"].to_numpy()
  #dummis_grado = pd.get_dummies(programa_larga.Grade, prefix = "Grado_", drop_first=True)
  datos_p = datos_grado.drop(columns = ["Grade", "resultado"])
  #datos_p = pd.concat([dummis_grado, datos_p], axis = 1, sort=False)
  datos_p["const"] = 1
  reg_1 = sm.OLS(y, datos_p).fit()
  intervalos = reg_1.conf_int()
  params = reg_1.params
  resumen = pd.concat([params, intervalos], axis = 1)
  resumen.columns = ["coef", "inf", "sup"]
  resumen["Grado"] = g
  coefs.append(resumen)
resumen = pd.concat(coefs)
resumen


Interpreta estos coeficientes. ¿Qué coeficiente es el que interpretas de manera causal?

In [ ]:
coef_trata =  resumen[resumen.index == "tratamiento"]
(ggplot(coef_trata, aes("Grado", "coef", ymin="inf", ymax="sup")) +
  geom_hline(yintercept = 0) +
   geom_linerange(colour = "red") +
   geom_point(size = 3) + coord_flip())

¿Cuáles son tus conclusiones acerca de estos coeficientes?

Ahora considera que dentro de este estudio, se permitió a maestro(a)s escoger si complementaban sus materiales usuales o sustituían del todo su material con el programa de TV. ¿En qué cambia tus conclusiones? Argumenta que puedes seguir interpretando estos efectos como causales.